In [1]:
%cd "/home/jspijkervet/git/clmr/"
import sys
sys.path.append("../")
from utils import parse_args
from data import get_dataset

/home/jspijkervet/git/clmr


/home/jspijkervet/miniconda3/envs/clmr/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit
/home/jspijkervet/miniconda3/envs/clmr/lib/python3.7/site-packages/librosa/util/decorators.py:9: NumbaDeprecationWarning: An import was requested from a module that has moved location.
Import of 'jit' requested from: 'numba.decorators', please update to use 'numba.core.decorators' or pin to Numba version 0.48.0. This alias will not be present in Numba version 0.50.0.
  from numba.decorators import jit as optional_jit


In [2]:
args = parse_args("./config/config.yaml", addit=[])
args.world_size = 1
args.supervised = 0
args.dataset = "magnatagatune" # magnatagatune

In [3]:
args.transforms_pitch = 0.6
args.transforms_reverb = 0.6
args.transforms_filter = 0.8
args.transforms_delay = 0.3
args.workers = 16
# data loaders
(
    train_loader,
    train_dataset,
    val_loader,
    val_dataset,
    test_loader,
    test_dataset,
) = get_dataset(args, pretrain=True, download=args.download)

100%|██████████| 3923/3923 [00:00<00:00, 146456.62it/s]

Concatenating tracks for pre-training (to avoid positive samples in the negative samples batch)
[magnatagatune train]: Loaded 3923 audio segments


[magnatagatune valid]: Loaded 1825 audio segments
[magnatagatune test]: Loaded 5329 audio segments


In [4]:
args.encoder

'resnet18'

In [6]:
from modules.shortchunk_cnn import ShortChunkCNN_Res

encoder = ShortChunkCNN_Res(n_channels=128,
                sample_rate=args.sample_rate,
                n_fft=512,
                f_min=0.0,
                f_max=args.sample_rate/2,
                n_mels=128,
                n_class=50)

for idx, (x, y) in enumerate(train_loader):
    x0 = x[0]
    x1 = x[1]
    
    print(x0.shape)
    h = encoder(x0)
    

torch.Size([48, 1, 59049])
torch.Size([48, 1, 128, 231])
torch.Size([48, 1, 1, 128, 231])


ValueError: expected 4D input (got 5D input)

In [ ]:
%%timeit
import time
t1 = time.time()
for step, (x, y) in enumerate(train_loader):   
#     print(time.time()-t0)
    if step == 1:
        break
#     if step % 20 == 0:
#         print(step, "/", len(train_loader))
        
print("Total", time.time()-t1)

In [ ]:
import torch
import time
import torchaudio
from pathlib import Path
import os
t0 = time.time()
for idx, _ in enumerate(train_dataset.index):
    track_id, clip_id, segment, fp, label = train_dataset.index[idx]
    label = torch.FloatTensor(label)
    audio = train_dataset.get_audio(fp)
    for s in range(0, train_dataset.num_segments):
        start_idx = int(s) * args.audio_length
        new_fp = os.path.splitext(fp)[0] + "-" + str(s) + ".wav"
        segment = audio[start_idx : start_idx + args.audio_length].reshape(1,-1)
        torchaudio.save(new_fp, torch.from_numpy(segment), sample_rate=args.sample_rate)
        
    if idx % 20 == 0:
        print(idx, "/", len(train_dataset))
    
print(time.time() - t0)

In [ ]:
%time train_dataset.get_audio("./datasets/magnatagatune/processed/train/1-22050.wav")